In [ ]:
from pyomo.environ import *
import numpy as np
from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models.pressure_changer import Pump
from idaes.generic_models.properties import iapws95
from idaes.generic_models.properties.helmholtz.helmholtz import (PhaseType)
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.core.util.scaling as iscale
import idaes.logger as idaeslog
import pytest

solver = SolverFactory('ipopt')
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = iapws95.Iapws95ParameterBlock(default={"phase_presentation":PhaseType.L})
m.fs.unit = Pump (default={
    "property_package": m.fs.properties,
})

In [ ]:
Q = 25.26
Tin = 333.15  # K
Pin = 122003.6  # Pa
Pdisc = (-3.1795*Q**4)+(150.43*Q**3)+(-2258.8*Q**2)+(-22975*Q)+(3000000)
eff =  (0.00002*Q**3)+(-0.0022*Q**2)+(0.0774*Q)+(0.009)

In [ ]:
hin = value(iapws95.htpx(Tin*units.K, Pin*units.Pa))
# set inputs
m.fs.unit.inlet.enth_mol[0].fix(hin)
m.fs.unit.inlet.pressure[0].fix(Pin)
m.fs.unit.inlet.flow_mol[0].fix(Q)
m.fs.unit.outlet.pressure[0].fix(Pdisc)
m.fs.unit.efficiency_pump.fix(eff)

In [ ]:
m.fs.unit.initialize(outlvl=idaeslog.INFO)

In [ ]:
opt = SolverFactory('ipopt') # set solver 
solve_status = opt.solve(m , tee=True) # set the solver status

In [ ]:
Qin_t = value (m.fs.unit.inlet.flow_mol[0])
print (Qin_t)

In [ ]:
m.fs.unit.inlet.flow_mol[0].unfix()
m.fs.unit.outlet.pressure[0].unfix()
m.fs.unit.efficiency_pump.unfix()

In [ ]:
m.fs.unit.bhp = Var(
    m.fs.unit.flowsheet().time, 
    initialize=1.0,
    doc="Break Horse Power",
    units="power",
)

m.fs.unit.freq = Var(
    m.fs.unit.flowsheet().time, 
    initialize = 55,
    #    m.frec,
    bounds = (55,60),
    doc="Pump frequency",
    units="hz",
)

In [ ]:
# N = np.array([n for n in range(55, 61)])
# m.frec = Set (initialize=N)

In [ ]:
# @m.fs.unit.Constraint(m.fs.config.time)
# def flow_calculation(unit,t):
#    return unit.inlet.flow_mol[t] == (
#        (Qin_t/60)*(m.fs.unit.freq[t,i] for i in m.frec)
#    )

In [ ]:
@m.fs.unit.Constraint(m.fs.config.time)
def flow_calculation(unit,t):
    return unit.inlet.flow_mol[t] == (
        (Qin_t/60)*(m.fs.unit.freq[t])
    )

@m.fs.unit.Constraint(m.fs.config.time)
def Pdesc_calculation_freq(unit, t):
    return unit.outlet.pressure[t] == (
        (-3.1795*Qin_t**4)+
        (150.43*Qin_t**3)+
        (-2258.8*Qin_t**2)+
        (-22975*Qin_t)+
        (3000000)*((m.fs.unit.freq[t]/60)**3)
    )

@m.fs.unit.Constraint(m.fs.config.time)
def eff_calculation_freq(unit, t):
    return unit.efficiency_pump[t] == (
        (0.00002*Qin_t**3)+
        (-0.0022*Qin_t**2)+
        (0.0774*Qin_t)+
        (0.009)
    )

@m.fs.unit.Constraint(m.fs.config.time, doc="Break horse power constraint")
def bhp_calculation_freq(unit, t):
    return unit.bhp[t] == (
        (-0.021*Qin_t**3)+
        (0.9103*Qin_t**2)+
        (0.0248*Qin_t)+
        (408.11)*((m.fs.unit.freq[t]/60)**2)
    )


In [ ]:
m.fs.objective = Objective(expr=m.fs.unit.bhp[0])

In [ ]:
m.fs.unit.inlet.flow_mol[0].setlb(22)
m.fs.unit.inlet.flow_mol[0].setub(26)
m.fs.unit.efficiency_pump.setlb(0.7)
m.fs.unit.efficiency_pump.setub(0.85)

In [ ]:
iscale.set_scaling_factor(m.fs.unit.control_volume.work, 1e-2)

In [ ]:
m.fs.unit.initialize(outlvl=idaeslog.INFO)

In [ ]:
opt = SolverFactory('ipopt') # set solver 
solve_status = opt.solve(m , tee=True) # set the solver status

In [ ]:
m.fs.unit.report()

In [ ]:
print (value(m.fs.unit.freq[0]))
print (value(m.fs.objective)

In [ ]:
#iscale.set_scaling_factor(m.fs.unit.control_volume.work, 1e-2, overwrite = True)
#iscale.constraint_scaling_transform (m.fs.flow_up_pdisc, 1e-5, overwrite = True)
#iscale.constraint_scaling_transform (m.fs.flow_low_pdisc, 1e-5, overwrite = True)
#iscale.calculate_scaling_factors(m)

In [ ]:
#m.fs.flow_up_freq = Constraint (expr= m.fs.unit.inlet.flow_mol[0] <= 27) 
#m.fs.flow_low_freq = Constraint (expr= m.fs.unit.inlet.flow_mol[0] >= 22) 
#m.fs.flow_up_pdisc = Constraint (expr= m.fs.unit.outlet.pressure[0] <= 2200000) 
#m.fs.flow_low_pdisc = Constraint (expr= m.fs.unit.outlet.pressure[0] >= 1400000) 
#m.fs.flow_up_eff = Constraint (expr= m.fs.unit.efficiency_pump[0] <= 0.85) 
#m.fs.flow_low_eff = Constraint (expr= m.fs.unit.efficiency_pump[0] >= 0.7) 

In [ ]:
#m.fs.flow_up_freq = Constraint (expr= m.fs.unit.control_volume.properties_out[t].flow_mol <= 27) 
#m.fs.flow_low_freq = Constraint (expr= m.fs.unit.control_volume.properties_out[t].flow_mol >= 22) 
#m.fs.flow_up_pdisc = Constraint (expr= m.fs.unit.control_volume.properties_out[t].pressure <= 1800000) 
#m.fs.flow_low_pdisc = Constraint (expr= m.fs.unit.control_volume.properties_out[t].pressure >= 1400000) 

In [ ]:
# @m.fs.unit.Constraint(m.fs.config.time)
#def frec_calculation(unit,t):
#    return m.fs.unit.freq [t] == (
#        (m.fs.unit.control_volume.properties_out[t].flow_mol*60)/(Qin_t)
#    )

#@m.fs.unit.Constraint(m.fs.config.time)
#def flow_calculation(unit,t):
#    return unit.inlet.flow_mol[t] == (
#        (Qin_t/60)*(m.fs.unit.freq[t])
#    )